#Imports

In [ ]:
!pip install transformers
!pip install evaluate
!pip install rouge


import torch
import json
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
import nltk
import spacy
import string
import evaluate  # Bleu
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pandas as pd
import numpy as np
import transformers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

from transformers import RealmConfig, RealmEmbedder, AutoTokenizer, RealmKnowledgeAugEncoder, RealmScorer


import warnings
warnings.filterwarnings("ignore")

# REALM

In [ ]:
import pandas as pd
df = pd.read_csv("data_withNeg1k-5Cols.csv")
df = df.dropna()

In [ ]:
#input_texts = ["How are you?", "Hi", "Hello"]
#candidates_texts = ["A cute cat.", "Nice to meet you!"]

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm

candidates_texts = []
input_texts = []

for index, row in df.iterrows():
    sentences = []
    input_texts.append(row['question'])
    sentences.append(row['context'])
    sentences.append(row['neg_context0'])
    sentences.append(row['neg_context1'])
    sentences.append(row['neg_context2'])
    sentences.append(row['neg_context3'])
    sentences.append(row['neg_context4'])
    candidates_texts.append(sentences)

print(len(input_texts))
print(len(candidates_texts[0]))

In [ ]:
import torch
from transformers import AutoTokenizer, RealmScorer

tokenizer = AutoTokenizer.from_pretrained("google/realm-cc-news-pretrained-scorer")
model = RealmScorer.from_pretrained("google/realm-cc-news-pretrained-scorer", num_candidates=6)

relevance_scores = []

for idx in range(input_texts):
  input_trunc = ' '.join(input_texts[idx].split()[:512])
  candidates_trunc = [' '.join(cand.split()[:512]) for cand in candidates_texts[idx][:]]
  inputs = tokenizer(input_trunc, max_length=512, padding="max_length", truncation=True, pad_to_max_length=True, return_tensors="pt")
  candidates_inputs = tokenizer.batch_encode_candidates(candidates_trunc, max_length=512, padding="max_length", truncation=True, pad_to_max_length=True, return_tensors="pt")

  outputs = model(
      **inputs,
      candidate_input_ids=candidates_inputs.input_ids,
      candidate_attention_mask=candidates_inputs.attention_mask,
      candidate_token_type_ids=candidates_inputs.token_type_ids,
  )
  relevance_scores.append(outputs.relevance_score)

# Loss Analysis

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv("data_withNeg2k_realm.csv")
df = df.dropna()

In [ ]:
df.loc[0]

In [ ]:
wrong_preds = df[(df['relevantContext'] > 0)]
len(wrong_preds)


lengths = [900, 1000, 1100, 1000, 1200, 1300, 1350, 1400, 1450]
wrongs = [len(wrong_preds[wrong_preds['n_tokens'] > l])/len(df[df['n_tokens'] > l]) for l in lengths]

# sns.histplot(y=wrongs, x=lengths)
# plt.title('Wrong predictions vs context length')

fig = plt.subplots(figsize = (8, 5))

# creating the bar plot
plt.bar(lengths, wrongs, width=40)

plt.xlabel("Length of context")
plt.ylabel("Proportion of wrong predictions")
plt.title("Wrong predictions vs context length")
plt.show()

In [ ]:
fig = plt.subplots(figsize = (8, 5))

lengths = [100, 200, 300, 400, 500, 600, 700, 800]
wrongs = [len(wrong_preds[wrong_preds['token_length_answer'] > l])/len(df[df['token_length_answer'] > l]) for l in lengths]

plt.title('Wrong predictions vs answer length')

# sns.histplot(y=wrongs, x=lengths)

# creating the bar plot
plt.bar(lengths, wrongs, width=40)

plt.xlabel("Length of answer")
plt.ylabel("Proportion of wrong predictions")
plt.show()